# This notebook demonstrates how the coco segmentation was generated for the odFridgeObjects dataset

Load in the object detection annotations and the jsonl generated from the instance segmentation notebook.

In [ ]:
import json
import os

with open(
    "../automl-image-object-detection-task-fridge-items/odFridgeObjects_coco.json"
) as coco_f:
    coco_data = json.load(coco_f)

with open("data/odFridgeObjectsMask/annotations_voc.jsonl") as voc_jsonl:
    jsonl = list(voc_jsonl)

## Generate COCO annotations with polygon format

In [ ]:
for line in jsonl:
    entry = json.loads(line)
    img_path = entry["image_url"]
    img_name = os.path.basename(img_path).split("/")[-1]
    img_id = os.path.splitext(img_name)[0]
    img_annotations = [x for x in coco_data["annotations"] if x["image_id"] == img_id]
    width = entry["image_details"]["width"]
    height = entry["image_details"]["height"]
    for idx, label in enumerate(entry["label"]):
        polygons = label["polygon"]
        segmentations = []
        for polygon in polygons:
            segmentation = []
            for id, vertex in enumerate(polygon):
                if (id % 2) == 0:
                    # x-coordinates (even index)
                    x = vertex * width
                    segmentation.append(round(x))

                else:
                    y = vertex * height
                    segmentation.append(round(y))
            segmentations.append(segmentation)
        img_annotations[idx]["segmentation"] = segmentations

outjson = "odFridgeObjectsMask_coco.json"
with open(outjson, "w") as outf:
    json.dump(coco_data, outf, indent=2)

## Generate coco annotations with RLE Format

In [ ]:
from azureml.automl.dnn.vision.object_detection.common import masktools
import torch

for line in jsonl:
    entry = json.loads(line)
    img_path = entry["image_url"]
    img_name = os.path.basename(img_path).split("/")[-1]
    img_id = os.path.splitext(img_name)[0]
    img_annotations = [x for x in coco_data["annotations"] if x["image_id"] == img_id]
    width = entry["image_details"]["width"]
    height = entry["image_details"]["height"]
    for idx, label in enumerate(entry["label"]):
        polygons = label["polygon"]
        segmentations = []
        for polygon in polygons:
            segmentation = []
            for id, vertex in enumerate(polygon):
                if (id % 2) == 0:
                    # x-coordinates (even index)
                    x = vertex * width
                    segmentation.append(round(x))

                else:
                    y = vertex * height
                    segmentation.append(round(y))
            segmentations.append(segmentation)
        rle_masks = masktools.convert_polygon_to_rle_masks(segmentations, height, width)
        binary_mask = masktools.decode_rle_masks_as_binary_mask(rle_masks)
        rle_mask = masktools.encode_mask_as_rle(torch.from_numpy(binary_mask))
        img_annotations[idx]["segmentation"] = rle_mask

outjson = "odFridgeObjectsMask_coco_rle.json"
with open(outjson, "w") as outf:
    json.dump(coco_data, outf, indent=2)